# Exercise 1

## First Part
- The exercise consists of implementing three similarity measures based on WordNet:
    - WU & Palmer similarity
    - Shortest Path similarity
    - Leacock & Chodorow similarity
- For each of these similarity measures, calculate Spearman's correlation indices and Pearson's correlation indices between the results obtained and the 'target' results in the dataset.

### Necessary imports and dataset loading

In [123]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus.reader.wordnet import Synset
nltk.download('wordnet')
import pandas as pd
import math
from scipy import stats as st

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lores\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [103]:
# Reading the file
df = pd.read_csv('dataset\combined.csv')
display(df)

,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62
...,...,...,...
348,shower,flood,6.03
349,weather,forecast,8.34
350,disaster,area,6.25
351,governor,office,6.34


### Support functions

In [104]:
# Defining the function to get the max depth of the synset

def max_depth_synset(synset: Synset) -> int:
    return max(len(path) for path in synset.hypernym_paths())

In [105]:
# Defining the function to get the least common subsumer of two synsets

def LCS(syn1: Synset, syn2: Synset) -> Synset | None:
    path1 = syn1.hypernym_paths()
    path2 = syn2.hypernym_paths()
    result = []

    # We loop through the paths of the first synset
    for pat1 in path1:
        x = pat1
        x.reverse()
        found = False
        for syn in x:
            if(not found):
                # We loop through the paths of the second synset
                for pat2 in path2:
                    if(syn in pat2):
                        result.append(syn)
                        found = True
                        break
            else:
                break
    if not result:
            return None
    else:
         max_depth = max(max_depth_synset(s) for s in result)
         max_hypernym = max(s for s in result if max_depth_synset(s) == max_depth)
         return max_hypernym

In [106]:
def path_len(syn1: Synset, syn2: Synset) -> int:
    path1 = syn1.hypernym_paths()
    path2 = syn2.hypernym_paths()
    result_syn1 = []
    result_syn2 = []

    common_ancestor = LCS(syn1,syn2)

    if(common_ancestor is None):
        return None
    else:
        for pat1 in path1:
            if(common_ancestor in pat1):
                result_syn1.append(pat1.index(syn1)-pat1.index(common_ancestor))
        for pat2 in path2:
            if(common_ancestor in pat2):
                result_syn2.append(pat2.index(syn2)-pat2.index(common_ancestor))
        return min(result_syn1) + min(result_syn2)
  

In [107]:
depth_max = max(max(len(hyp_path) for hyp_path in ss.hypernym_paths()) for ss in wn.all_synsets())

### Wu Palmer

$$ LCS(s1, s2) = {2 * depth(LCS(s1,s2)) \over depth(s1) + depth(s2)} $$

In [108]:
def wu_palmer(syn1: Synset, syn2: Synset) -> float:
    # We get all the synsets of the two words
    depht1 = max_depth_synset(syn1)
    depht2 = max_depth_synset(syn2)
    # Finding the least common subsumer of the two synsets
    common_ancestor = LCS(syn1,syn2)
    if common_ancestor is None:
        return 0
    return 2 * max_depth_synset(common_ancestor) / ( depht1 + depht2 )

### Shortest Path

$$ LCS(s1, s2) = 2 * depthMax - len(s1,s2) $$

In [109]:
def shortest_path(syn1: Synset, syn2: Synset) -> int:
    pat = path_len(syn1,syn2)
    if pat is None:
        return 0
    elif (pat == 0):
        return 2 * depth_max
    elif (pat == depth_max):
        return 0
    else:
        return 2 * depth_max - pat

### Leakcock & Chodorow

$$ sim_LC(s1, s2) = -log(\frac{len(s1,s2)}{2*depthMax}) $$

In [110]:
def leakcock_chodorow (syn1: Synset, syn2: Synset) -> float:
    pat = path_len(syn1,syn2)
    if pat is None:
        return 0
    elif (pat == 0):
        return -math.log((1)/(2 * depth_max + 1))
    else:
        return -math.log((pat)/(2 * depth_max))

### Cycle for every synset of the words

In [118]:
def synsets_cycle(word1: str, word2: str, metric: callable) -> float | str:
    syn1 = wn.synsets(word1)
    syn2 = wn.synsets(word2)
    result = []
    for s1 in syn1:
        for s2 in syn2:
            result.append(metric(s1,s2))
    if not result:
        return 0
    else:
        return max(result)

In [112]:
# Test
print("WU Palmer between dog and cat:", synsets_cycle('dog','cat', wu_palmer))
print("WU Palmer between stock and live:", synsets_cycle('stock','live', wu_palmer))
print("Shortest Path between Jerusalem and Palestinian:", synsets_cycle('Jerusalem','Palestinian', shortest_path))
print("Leakcock Chodorow between dog and cat:", synsets_cycle('dog','cat', leakcock_chodorow))

WU Palmer between dog and cat: 0.8571428571428571
WU Palmer between stock and live: 0.2857142857142857
Shortest Path between Jerusalem and Palestinian: 24
Leakcock Chodorow between dog and cat: 2.3025850929940455


### Loop over all pair of words in the dataset and i print the three similarity measures

In [143]:
# Loop through the dataset and calculate the similarity for each pair of words using the three metrics and i add the result to a new dataframe
df_metrics = pd.DataFrame(columns=['Word 1', 'Word 2', "Human (mean)", "WU Palmer", "Shortest Path", "Leakcock Chodorow"])
for index, row in df.iterrows():
    df_metrics.loc[len(df_metrics)] = {'Word 1': row['Word 1'], 'Word 2': row['Word 2'], "Human (mean)": row['Human (mean)'], "WU Palmer": synsets_cycle(row['Word 1'], row['Word 2'], wu_palmer) * 10, "Shortest Path": (synsets_cycle(row['Word 1'], row['Word 2'], shortest_path) /40) * 10, "Leakcock Chodorow": (synsets_cycle(row['Word 1'], row['Word 2'], leakcock_chodorow) / math.log(41)) * 10}
display(df_metrics)

,Word 1,Word 2,Human (mean),WU Palmer,Shortest Path,Leakcock Chodorow
0,love,sex,6.77,9.230769,9.75,9.933507
1,tiger,cat,7.35,9.655172,9.75,9.933507
2,tiger,tiger,10.00,10.000000,10.00,10.000000
3,book,paper,7.46,8.750000,9.50,8.066983
4,computer,keyboard,7.62,8.235294,9.25,6.975136
...,...,...,...,...,...,...
348,shower,flood,6.03,6.363636,9.00,6.200459
349,weather,forecast,8.34,1.333333,6.75,3.026547
350,disaster,area,6.25,5.000000,8.00,4.333935
351,governor,office,6.34,5.263158,7.75,4.016766


### Calculation of correlation with Spearman and Pearson metrics between the human similarity and the similarity calculated by the three metrics

In [161]:
# Calculate the correlation between the human similarity and the similarity calculated by the three metrics

print("-------------------------------------------")
print("WU Palmer Metric:")
spearman_correlation = st.spearmanr(df_metrics["WU Palmer"], df_metrics["Human (mean)"])
print("Spearman Correlation:", spearman_correlation)
if spearman_correlation[1] < 0.05:
    print("The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.\n")
else:
    print("The distribution of the WU Palmer metric is almost the same as the distribution of the Human Similarity.\n")
pearson_correlation = st.pearsonr(df_metrics["WU Palmer"], df_metrics["Human (mean)"])
print("Pearson Correlation:", pearson_correlation)
if pearson_correlation[1] < 0.05:
    print("The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.")
else:
    print("The distribution of the WU Palmer metric is almost the same as the distribution of the Human Similarity.")
print("-------------------------------------------")

print("Shortest Path Metric:")
spearman_correlation = st.spearmanr(df_metrics["Shortest Path"], df_metrics["Human (mean)"])
print("Spearman Correlation:", spearman_correlation)
if spearman_correlation[1] < 0.05:
    print("The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.\n")
else:
    print("The distribution of the Shortest Path metric is almost the same as the distribution of the Human Similarity.\n")
pearson_correlation = st.pearsonr(df_metrics["Shortest Path"], df_metrics["Human (mean)"])
print("Pearson Correlation:", pearson_correlation)
if pearson_correlation[1] < 0.05:
    print("The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.")
else:
    print("The distribution of the Shortest Path metric is almost the same as the distribution of the Human Similarity.")
print("-------------------------------------------")

print("Leakcock Chodorow Metric:")
spearman_correlation = st.spearmanr(df_metrics["Leakcock Chodorow"], df_metrics["Human (mean)"])
print("Spearman Correlation:", spearman_correlation)
if spearman_correlation[1] < 0.05:
    print("The distribution of the Leakcock Chodorow metric is significantly different from the distribution of the Human Similarity.\n")
else:
    print("The distribution of the Leakcock Chodorow metric is almost the same as the distribution of the Human Similarity.\n")
pearson_correlation = st.pearsonr(df_metrics["Leakcock Chodorow"], df_metrics["Human (mean)"])
print("Pearson Correlation:", pearson_correlation)
if pearson_correlation[1] < 0.05:
    print("The distribution of the Leakcock Chodorow metric is significantly different from the distribution of the Human Similarity.")
else:
    print("The distribution of the Leakcock Chodorow metric is almost the same as the distribution of the Human Similarity.")
print("-------------------------------------------")

-------------------------------------------
WU Palmer Metric:
Spearman Correlation: SignificanceResult(statistic=0.34531446776235897, pvalue=2.5376440594705738e-11)
The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.

Pearson Correlation: PearsonRResult(statistic=0.28732609798215053, pvalue=3.895408936985221e-08)
The distribution of the WU Palmer metric is significantly different from the distribution of the Human Similarity.
-------------------------------------------
Shortest Path Metric:
Spearman Correlation: SignificanceResult(statistic=0.2883010732370922, pvalue=3.4876313366885126e-08)
The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.

Pearson Correlation: PearsonRResult(statistic=0.16411938908972765, pvalue=0.0019775111177550756)
The distribution of the Shortest Path metric is significantly different from the distribution of the Human Similarity.
---

## Second Part

- Implementing Lesk's algorithm (from NLTK).
    1. Extract 50 sentences from the SemCor corpus (corpus annotated with WN synsets) and disambiguate (at least) one noun per sentence. Calculate the accuracy of the implemented system based on the senses annotated in SemCor.
    SemCor is available at the URL: http://web.eecs.umich.edu/~mihalcea/downloads.html
    2. Randomize the selection of the 50 sentences and the selection of the term to be disambiguated, and return the average accuracy over (for example) 10 runs of the program.
- [OPTIONAL]: implement corpus-Lesk algorithm using Sem-Cor